# Fraud Detection with Graph databases and Machine Learning

## Importing the required Python libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


## Loading and exploring the banksim dataset 

In [2]:
banksim_df = pd.read_csv("../data/bs140513_032310.csv")
banksim_df.head()

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud
0,0,'C1093826151','4','M','28007','M348934600','28007','es_transportation',4.55,0
1,0,'C352968107','2','M','28007','M348934600','28007','es_transportation',39.68,0
2,0,'C2054744914','4','F','28007','M1823072687','28007','es_transportation',26.89,0
3,0,'C1760612790','3','M','28007','M348934600','28007','es_transportation',17.25,0
4,0,'C757503768','5','M','28007','M348934600','28007','es_transportation',35.72,0


In [3]:
banksim_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 594643 entries, 0 to 594642
Data columns (total 10 columns):
step           594643 non-null int64
customer       594643 non-null object
age            594643 non-null object
gender         594643 non-null object
zipcodeOri     594643 non-null object
merchant       594643 non-null object
zipMerchant    594643 non-null object
category       594643 non-null object
amount         594643 non-null float64
fraud          594643 non-null int64
dtypes: float64(1), int64(2), object(7)
memory usage: 45.4+ MB


Viewing the split of the output classes - fraudulent and genuine transactions

In [4]:
banksim_df['fraud'].value_counts()

0    587443
1      7200
Name: fraud, dtype: int64

In [5]:
# Number of unique values per column in the banksim dataset
banksim_df.nunique()

step             180
customer        4112
age                8
gender             4
zipcodeOri         1
merchant          50
zipMerchant        1
category          15
amount         23767
fraud              2
dtype: int64

## Preprocessing the data 

In [6]:
# Obtaining the number of null values in each column
banksim_df.isna().sum()

step           0
customer       0
age            0
gender         0
zipcodeOri     0
merchant       0
zipMerchant    0
category       0
amount         0
fraud          0
dtype: int64

Looks like there are no null values in the dataset.

In [7]:
# Retrieving the class attribute from the dataframe
labels = banksim_df['fraud']

In [8]:
'''
Removing unwanted columns
Since zipcodeOri and zipMerchant have the same value for all the rows, these columns are redundant
'''

feature_df = banksim_df.drop(['step', 'customer', 'zipcodeOri', 'zipMerchant', 'fraud'], axis=1)

In [9]:
feature_df.head()

,age,gender,merchant,category,amount
0,'4','M','M348934600','es_transportation',4.55
1,'2','M','M348934600','es_transportation',39.68
2,'4','F','M1823072687','es_transportation',26.89
3,'3','M','M348934600','es_transportation',17.25
4,'5','M','M348934600','es_transportation',35.72


In [10]:
# One hot encoding the categorical variables
feature_df = pd.get_dummies(feature_df, columns=['age', 'gender', 'category', 'merchant'])
feature_df.head()

,amount,age_'0',age_'1',age_'2',age_'3',age_'4',age_'5',age_'6',age_'U',gender_'E',...,merchant_'M732195782',merchant_'M78078399',merchant_'M840466850',merchant_'M855959430',merchant_'M857378720',merchant_'M85975013',merchant_'M923029380',merchant_'M933210764',merchant_'M97925176',merchant_'M980657600'
0,4.55,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,39.68,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,26.89,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,17.25,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,35.72,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# Standardizing the features
standard_scaler = StandardScaler()
scaled_df = pd.DataFrame(standard_scaler.fit_transform(feature_df), columns = feature_df.columns)

scaled_df.head()

,amount,age_'0',age_'1',age_'2',age_'3',age_'4',age_'5',age_'6',age_'U',gender_'E',...,merchant_'M732195782',merchant_'M78078399',merchant_'M840466850',merchant_'M855959430',merchant_'M857378720',merchant_'M85975013',merchant_'M923029380',merchant_'M933210764',merchant_'M97925176',merchant_'M980657600'
0,-0.299276,-0.064347,-0.329165,-0.678119,-0.573390,2.110495,-0.343144,-0.217136,-0.044553,-0.044553,...,-0.031992,-0.052072,-0.048562,-0.10179,-0.014325,-0.214919,-0.023313,-0.010773,-0.031754,-0.054624
1,0.016067,-0.064347,-0.329165,1.474668,-0.573390,-0.473822,-0.343144,-0.217136,-0.044553,-0.044553,...,-0.031992,-0.052072,-0.048562,-0.10179,-0.014325,-0.214919,-0.023313,-0.010773,-0.031754,-0.054624
2,-0.098742,-0.064347,-0.329165,-0.678119,-0.573390,2.110495,-0.343144,-0.217136,-0.044553,-0.044553,...,-0.031992,-0.052072,-0.048562,-0.10179,-0.014325,-0.214919,-0.023313,-0.010773,-0.031754,-0.054624
3,-0.185275,-0.064347,-0.329165,-0.678119,1.744015,-0.473822,-0.343144,-0.217136,-0.044553,-0.044553,...,-0.031992,-0.052072,-0.048562,-0.10179,-0.014325,-0.214919,-0.023313,-0.010773,-0.031754,-0.054624
4,-0.019480,-0.064347,-0.329165,-0.678119,-0.573390,-0.473822,2.914227,-0.217136,-0.044553,-0.044553,...,-0.031992,-0.052072,-0.048562,-0.10179,-0.014325,-0.214919,-0.023313,-0.010773,-0.031754,-0.054624


## Training supervised learning models using intrinsic features from the dataset

In [12]:
k_fold = KFold(n_splits=5, random_state=None, shuffle=False)


random_forest = RandomForestClassifier(max_depth=4, n_estimators=150)
svm = SVC(gamma="auto")

scaled_df = scaled_df.values
labels = labels.values

In [13]:


for train_index, test_index in k_fold.split(scaled_df):
    X_train, X_test = scaled_df[train_index], scaled_df[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    
    clf = random_forest.fit(X_train, y_train)

    predictions = clf.predict(X_test)
    accuracy = accuracy_score(y_test, predictions) * 100

    metrics = precision_recall_fscore_support(y_test, predictions, average='macro')
    precision, recall = metrics[0], metrics[1]
    f1_score = 2 * (precision * recall) / (precision + recall)

    
    print("Accuracy = {0:.4f}%".format(accuracy))
    print("Precision = {0:.4f}".format(precision))
    print("Recall = {0:.4f}".format(recall))
    print("F1 Score = {0:.4f}".format(f1_score))
    print("---------------------------------------------")

Accuracy = 98.9035%
Precision = 0.9695
Recall = 0.6267
F1 Score = 0.7612
---------------------------------------------
Accuracy = 99.0103%
Precision = 0.9802
Recall = 0.6094
F1 Score = 0.7516
---------------------------------------------
Accuracy = 99.0902%
Precision = 0.9860
Recall = 0.6118
F1 Score = 0.7551
---------------------------------------------
Accuracy = 99.1280%
Precision = 0.9825
Recall = 0.6114
F1 Score = 0.7537
---------------------------------------------
Accuracy = 99.1423%
Precision = 0.9938
Recall = 0.6019
F1 Score = 0.7498
---------------------------------------------


In [19]:
for train_index, test_index in k_fold.split(scaled_df):
    X_train, X_test = scaled_df[train_index], scaled_df[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    
    clf = svm.fit(X_train, y_train)
    print("Finished training")
    predictions = clf.predict(X_test)
    accuracy = accuracy_score(y_test, predictions) * 100

    metrics = precision_recall_fscore_support(y_test, predictions, average='macro')
    precision, recall = metrics[0], metrics[1]
    f1_score = 2 * (precision * recall) / (precision + recall)

    
    print("Accuracy = {0:.4f}%".format(accuracy))
    print("Precision = {0:.4f}".format(precision))
    print("Recall = {0:.4f}".format(recall))
    print("F1 Score = {0:.4f}".format(f1_score))
    print("---------------------------------------------")

Finished training
Accuracy = 99.0800%
Precision = 0.9000
Recall = 0.7828
F1 Score = 0.8373
---------------------------------------------
Finished training
Accuracy = 99.1100%
Precision = 0.9048
Recall = 0.7806
F1 Score = 0.8381
---------------------------------------------
Finished training
Accuracy = 99.4300%
Precision = 0.9509
Recall = 0.8363
F1 Score = 0.8899
---------------------------------------------
Finished training
Accuracy = 98.9400%
Precision = 0.8884
Recall = 0.7134
F1 Score = 0.7913
---------------------------------------------
Finished training
Accuracy = 99.2700%
Precision = 0.9606
Recall = 0.7881
F1 Score = 0.8658
---------------------------------------------
